In [ ]:
import pathlib
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
from pathlib import Path
from sklearn.model_selection import train_test_split
from keras import regularizers
import json

# Insert Dice Coefficient as another metric

import keras.backend as K
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Used later for Saving different Plots
def save_fig(fig_id,path, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(path, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Used to make a Test Set Patch to get Visual Predictions
def create_instances(test_folder):
    instances = []
    input_path =os.path.join(test_folder,"Input")
    input_names = os.listdir(input_path)
    for i in input_names:
        loaded_array_x = np.load(os.path.join(input_path,i),allow_pickle = True)
        instances.append(loaded_array_x)
    return np.array(instances),input_names

def get_paths(DATA_PATH,LOADCASE_FOLDERS):
    INPUT_PATHS = []
    OUTPUT_PATHS = []
# Iterate through the Loadcase folders
    for i in LOADCASE_FOLDERS:
# Get the Input/Output root for each of those loadcase folders
        INPUT_ROOTS = os.path.join(DATA_PATH,i,"Input")
        OUTPUT_ROOTS = os.path.join(DATA_PATH,i,"Output")
# Get the individual names of the inputs and outputs
# I didn't use the same names, since there might be a need to keep them seperated,
# if there is an Input but no output data, since there are not enough iterations and such
        input_names = os.listdir(INPUT_ROOTS)
        output_names = os.listdir(OUTPUT_ROOTS)
        for i in input_names:
            INPUT_PATHS.append(os.path.join(INPUT_ROOTS,i))
        for i in output_names:
            OUTPUT_PATHS.append(os.path.join(OUTPUT_ROOTS,i))
    COMBINED_PATHS = list(zip(INPUT_PATHS,OUTPUT_PATHS))
# Input is [0] Output is [1]
    return COMBINED_PATHS



# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
class DataGenerator(keras.utils.Sequence):
    def __init__(self, input_paths, output_paths, output_dim=(64,48), output_channels = 1,
                 batch_size=32, input_dim=(65, 49), input_channels=6, shuffle=True):
        self.batch_size = batch_size
#     Get the Output Information
        self.output_paths = output_paths
        self.output_dim = output_dim
        self.output_channels= output_channels
#    Get the Input Information
        self.input_paths = input_paths
        self.input_dim =input_dim
        self.input_channels = input_channels
#   Shuffle Parameter
        self.shuffle = shuffle
        self.on_epoch_end()
#   Number of batches per epoch
    def __len__(self):
        return int(np.floor(len(self.input_paths) / self.batch_size))
#   Generates and Loads the Data
#   With getitem we can access the data through indexing, instead of calling a method with number
#   In this case we access the whole minibatch
    def __getitem__(self, index):
#   Generate indexes of the batch, index gets decided randomly 
#   The self.indexes are randomly shuffled
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

#   Make temporary path lists from the indexes with len = Batch Size
        input_paths_temp = [self.input_paths[k] for k in indexes]
        output_paths_temp = [self.output_paths[k] for k in indexes]

#    Call the data generation method(in this case load)
        X, y = self.__data_generation(input_paths_temp,output_paths_temp)

        return X, y
#   Shuffles the indexes after each epoch
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.input_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self,input_paths_temp,output_paths_temp):
#    Placeholders --- necessary?
        X = np.empty((self.batch_size, self.input_channels, *self.input_dim), dtype= float)
        y = np.empty((self.batch_size, self.output_channels,*self.output_dim), dtype=float)

        for i,path in enumerate(input_paths_temp):
            loaded_array_x = np.load(path,allow_pickle=True)
            X[i,] = loaded_array_x
        for i,path in enumerate(output_paths_temp):
            loaded_array_y = np.load(path,allow_pickle=True)
            y[i,] = loaded_array_y
         

        return X, y
    


# Create the Model

def create_unet(IMG_HEIGHT_INPUT,IMG_WIDTH_INPUT,IMG_CHANNELS_INPUT,
                IMG_CHANNELS_OUTPUT,l2_1 = 0.0,l2_2=0.0):

    def encoding_block(input_,depth, l2_ = 0.0):
        BN_1 = layers.BatchNormalization(axis=1)(input_)
        conv_1= layers.Conv2D(depth, (3, 3), activation='relu',
                              kernel_regularizer = regularizers.l2(l2_),
                              # bias_regularizer=regularizers.l2(l2_),
                              data_format='channels_first',padding="same")(BN_1)
        BN_2 = layers.BatchNormalization(axis=1)(conv_1)
        output_ = layers.MaxPool2D(pool_size=2,strides=2,data_format='channels_first')(BN_2)
        return output_
    def decoding_block(input_,depth,l2_ = 0.0):
        Trans_1= layers.Conv2DTranspose(depth, (2,2),strides=(2, 2), 
                              kernel_regularizer = regularizers.l2(l2_),
                              # bias_regularizer=regularizers.l2(l2_),
                                data_format='channels_first')(input_)
        BN_1 = layers.BatchNormalization(axis=1)(Trans_1)
        conv_1= layers.Conv2D(depth, (7, 7), activation='relu',
                              kernel_regularizer = regularizers.l2(l2_),
                              # bias_regularizer=regularizers.l2(l2_),
                              data_format='channels_first',padding="same")(BN_1)
        output_ = layers.BatchNormalization(axis=1)(conv_1)
        return output_
    def bottleneck (input_,depth,l2_ = 0.0):
        conv_1 =layers.Conv2D(depth, (7, 7), activation='relu',
                              kernel_regularizer = regularizers.l2(l2_),
                              # bias_regularizer=regularizers.l2(l2_),
                              data_format='channels_first',padding="same")(input_)
        BN_1= layers.BatchNormalization(axis=1)(conv_1)
        conv_2 =layers.Conv2D(depth, (7, 7), activation='relu',
                              kernel_regularizer = regularizers.l2(l2_),
                              # bias_regularizer=regularizers.l2(l2_),
                              data_format='channels_first',padding="same")(BN_1)
        output_ = layers.BatchNormalization(axis=1)(conv_2)
        return output_

    inputs = layers.Input((IMG_CHANNELS_INPUT, IMG_HEIGHT_INPUT, IMG_WIDTH_INPUT)) 
    conv_0a = layers.Conv2D(32, (2, 2), activation='relu',
                              kernel_regularizer = regularizers.l2(l2_1),
                              # bias_regularizer=regularizers.l2(l2_1),
                            data_format='channels_first',padding="valid")(inputs)
    BN_1 = layers.BatchNormalization(axis=1)(conv_0a)
    conv_enc1 =  layers.Conv2D(64, (3, 3), activation='relu',
                              kernel_regularizer = regularizers.l2(l2_1),
                              # bias_regularizer=regularizers.l2(l2_1),
                            data_format='channels_first',padding="same")(BN_1)
    enc_1 = encoding_block(conv_enc1,64,l2_1)
    conv_enc2 =  layers.Conv2D(128, (3, 3), activation='relu',
                              kernel_regularizer = regularizers.l2(l2_2),
                              # bias_regularizer=regularizers.l2(l2_2),
                            data_format='channels_first',padding="same")(enc_1)
    enc_2 = encoding_block(conv_enc2,128,l2_2)
    conv_enc3 =  layers.Conv2D(256, (3, 3), activation='relu',
                              kernel_regularizer = regularizers.l2(l2_2),
                              # bias_regularizer=regularizers.l2(l2_2),
                            data_format='channels_first',padding="same")(enc_2)
    enc_3 = encoding_block(conv_enc3,256,l2_2)
    bottleneck_ = bottleneck(enc_3,256,l2_2)
    dec_1 = decoding_block(bottleneck_,128,l2_2)
    u_1 = layers.concatenate([dec_1,enc_2],axis=1)
    dec_2 = decoding_block(u_1,64,l2_1)
    u_2 = layers.concatenate([dec_2,enc_1],axis=1)
    dec_3 = decoding_block(u_2,32,l2_1)
    u_3 = layers.concatenate([dec_3,BN_1],axis=1)
    conv_e1 =layers.Conv2D(32, (7, 7), activation='relu',
                                kernel_regularizer = regularizers.l2(l2_1),
                              # bias_regularizer=regularizers.l2(l2_1),
                              data_format='channels_first',padding="same")(u_3)
    BN_e1 = layers.BatchNormalization(axis=1)(conv_e1)
    conv_e2 =layers.Conv2D(32, (7, 7), activation='relu',
                              kernel_regularizer = regularizers.l2(l2_1),
                              # bias_regularizer=regularizers.l2(l2_1),
                              data_format='channels_first',padding="same")(BN_e1)
    BN_e2 = layers.BatchNormalization(axis=1)(conv_e2)
    outputs = layers.Conv2D(IMG_CHANNELS_OUTPUT, (1, 1), activation='sigmoid',
                                     data_format='channels_first',padding="same")(BN_e2)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model


In [ ]:
# Train multiple models in a Loop, with example IDs and values

ID_LOOP = ["ID_319","ID_320","ID_321"]
BATCH_SIZE_LOOP = [32,64,128]
l2_1 = 1e-7
l2_2 = 2e-7

for i,s in enumerate(ID_LOOP):

    random_seed = 42
    np.random.seed = random_seed
    TRIAL_ID = s
    # The Paths of the Training Data
    DATA_ROOT = "/kaggle/input/rotated-dataset/"
    DATA_ID = "split_rot"
    DATA_PATH  = os.path.join(DATA_ROOT,DATA_ID)
    LOADCASE_FOLDERS = os.listdir(DATA_PATH)

    # Define the Roots for the Callpacks and more (TRIAL ID dependant path)
    SAVING_ROOT= "/kaggle/working/TRIALS"
    TRIAL_PATH = os.path.join(SAVING_ROOT,TRIAL_ID)
    os.makedirs(TRIAL_PATH,exist_ok=True)

    # Tensorboard Path
    TENSORBOARD_PATH = os.path.join(TRIAL_PATH,"tensorboard")
    os.makedirs(TENSORBOARD_PATH,exist_ok=True)

    # Checkpoint Path
    CHECKPOINT_PATH = os.path.join(TRIAL_PATH,"Model_Checkpoint")
    text="checkpoint.hdf5"
    cp_path = os.path.join(TRIAL_PATH,text)

    # Define the Paths to save the models to
    models_dir = os.path.join(TRIAL_PATH,"models")
    os.makedirs(models_dir,exist_ok=True)
    name = TRIAL_ID + ".h5"

    # History Saving Path
    title = str ("history_"+TRIAL_ID)
    HISTORY_PATH =os.path.join(TRIAL_PATH,title)

    # Test Set Path to Visualize the Predictions 
    TEST_SET_PATH = "/kaggle/input/rotated-test-set/test_set_rot"

    # Define the Dimensions of the In and Output
    IMG_HEIGHT_INPUT= 49
    IMG_WIDTH_INPUT = 65
    IMG_CHANNELS_INPUT= 6
    IMG_HEIGHT_OUTPUT = 48
    IMG_WIDTH_OUTPUT = 64
    IMG_CHANNELS_OUTPUT = 1


    # Regularizer Parameters

    l2_1 = l2_1
    l2_2 = l2_2


    # Training Hyperparameters

    BATCH_SIZE_ = BATCH_SIZE_LOOP[i]
    shuffle_data_ = True
    test_set_fraction = 0.33
    loss_ = "binary_crossentropy"
    optimizer_ = keras.optimizers.Adam()
    metrics_ = ["binary_accuracy",dice_coef]
    epochs_ = 20
    
    parameters_generator = {'output_dim': (IMG_HEIGHT_OUTPUT,IMG_WIDTH_OUTPUT),
                            'output_channels': IMG_CHANNELS_OUTPUT,
                            'batch_size': BATCH_SIZE_,
                            'input_dim': (IMG_HEIGHT_INPUT,IMG_WIDTH_INPUT),
                            'input_channels': IMG_CHANNELS_INPUT,
                            'shuffle': shuffle_data_
    }

    
    PATHS = get_paths(DATA_PATH,LOADCASE_FOLDERS)
    print("Number of found paths: "len(PATHS))
    
    # Create the Generators

    # Split the Data into Test and Valids
    X_paths_train,X_paths_valid,Y_paths_train,Y_paths_valid = train_test_split(
    # Have to unwrap the lists
        [path_[0] for path_ in PATHS], [path_[1] for path_ in PATHS], test_size=test_set_fraction,
        random_state=random_seed)

    training_generator = DataGenerator(X_paths_train, Y_paths_train, **parameters_generator)
    validation_generator = DataGenerator(X_paths_valid, Y_paths_valid, **parameters_generator)

    # Callbacks

    early_stop_cb= tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(TENSORBOARD_PATH)
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(cp_path, save_best_only=True,
                                                       monitor='val_loss'
                                                      )

    callbacks_ = [checkpoint_cb,tensorboard_cb]
    u_net = create_unet(IMG_HEIGHT_INPUT,IMG_WIDTH_INPUT,IMG_CHANNELS_INPUT,IMG_CHANNELS_OUTPUT,
                    l2_1,l2_1)
    u_net.compile(loss=loss_,
                       optimizer=keras.optimizers.Adam(),
                    metrics=metrics_)
    history = u_net.fit_generator(generator=training_generator,epochs = epochs_ ,callbacks=callbacks_,
                        validation_data=validation_generator,verbose=1)

    u_net.save(os.path.join(models_dir,name))

    
    # Save the History of the Loss and the Metrics

    # Get the dictionary containing each metric and the loss for each epoch
    history_dict = history.history
    # Save it under the form of a json file
    json.dump(str(history_dict), open(HISTORY_PATH, 'w'))
    
    plt.plot(history.history['binary_accuracy'])
    plt.plot(history.history['val_binary_accuracy'])
    plt.title('Model binary_accuracy')
    plt.ylabel('binary_accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    save_fig(TRIAL_ID+"_binary_accuracy",TRIAL_PATH)
    img_ = plt.show()

    plt.plot(history.history['dice_coef'])
    plt.plot(history.history['val_dice_coef'])
    plt.title('Model dice coefficent')
    plt.ylabel('dice coefficent')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    save_fig(TRIAL_ID+"_dice coefficent",TRIAL_PATH)
    img_ = plt.show()
    # Plot training & validation loss values

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    save_fig(TRIAL_ID+"_loss",TRIAL_PATH)
    img_ = plt.show()

